## BANK MARKETING

Projet réalisé par :
* Arnaud Leleu
* Camille Kienlen
* Clément Guillet
* Julien Musschoot

### INTRODUCTION AU PROJET

In [67]:
#!/usr/bin/env python3

# Import des librairies

#from IPython.display import display

#import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import RobustScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.compose import ColumnTransformer
#from sklearn.inspection import permutation_importance
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import neighbors
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool

from IPython.display import Markdown

from pathlib import Path

project_root = Path('.')
data_path = project_root / 'data'
build_path = project_root / 'build'
pages_path = project_root / 'pages'
projet_path = pages_path / '1-projet'
donnees_path = pages_path / '2-donnees'
visu_path = pages_path / '3-visu'
modelisation_path = pages_path / '4-modelisation'
ml_path = pages_path / '5-ml'
conclusion_path = pages_path / '6-conclusion'

# Exploitation d'un fichier .md pour la page de présentation du projet
txt_projet = open(projet_path / 'contexte_et_objectifs.md').read()

# Fichiers exploités sur la page de présentation du jeu de données
txt_cadre = open(donnees_path / 'cadre.md').read()
txt_pertinence = open(donnees_path / 'pertinence.md').read()
df_pertinence = pd.read_csv(donnees_path / "tableau_variables.csv", sep = ";", index_col=0, lineterminator="\n")
txt_conclusion_prepocess = open(donnees_path / 'conclusion_preprocess.md').read()

# Fichiers exploités pour la page de modélisation du projet
txt_classif_choix = open(modelisation_path / 'classification_choix.md').read()
txt_interpretation = open(modelisation_path / 'interpretation.md').read()


separation = "--------------------------------------------------------"

In [62]:
display(Markdown(txt_projet))

#### Contexte

##### Contexte d’insertion du projet dans votre métier ou intérêts professionnels

Le projet « bank marketing » nous présente en tant que data analystes pour une banque. La direction de la société nous demande de trouver les meilleures stratégies pour améliorer les prochaines campagnes marketing. Sur la base des informations collectées lors de la dernière campagne, nous devrons identifier les modèles les plus efficaces pour atteindre un objectif bien précis, le succès d’une future campagne de souscription.

Ce projet permet de conjuguer l’analyse avancée de données avec une compréhension approfondie d’enjeux économiques et financiers pour la banque. L’idée est d’améliorer les processus et de fournir à la direction des recommandations stratégiques visant à favoriser la réussite globale de l'entreprise.

##### Dimension Technique

**Utilisation de Python de A à Z :** Nous utiliserons Python au maximum des concepts que nous avons appris pendant notre formation, avec des bibliothèques telles que Pandas pour le traitement des données, Seaborn et Matplotlib pour la visualisation des résultats, et Scikit-learn pour l’analyse prédictive.

Nous articulerons notre progression autour de quatre thèmes principaux.

1. Préparation et nettoyage des données :
    * Traitement des valeurs manquantes : identifier et corriger les données manquantes pour éviter les biais dans le modèle (utilisation de médianes, moyennes, imputation des k proches…)
    * Encodage des variables catégorielles : convertir les variables qualitatives en formats numériques adaptés aux algorithmes de Machine Learning (one hot, label encoding, ordinal encoding)
    * Ajustement des variables : normaliser ou standardiser les variables pour assurer une cohérence dans la modélisation.
2. Analyse exploratoire des données :
    * Identification des schémas : analyser les distributions des données pour détecter des motifs récurrents et des tendances significatives.
    * Exploration des relations : utiliser des visualisations et des statistiques descriptives pour examiner les corrélations entre les variables explicatives et la variable cible (histogrammes, boxplots, heatmaps…).
3. Modélisation prédictive :
    * Sélection et entrainement des modèles : tester et entraîner divers algorithmes de Machine Learning pour déterminer le modèle le plus performant (forêts aléatoires, régressions logistiques, classificateur d’arbre de décision…).
    * Évaluation des performances : mesurer l'efficacité des modèles à l'aide de métriques telles que l’accuracy, la précision, les MAE et MSE, ou encore le f1_score, pour sélectionner le modèle optimal.
4. Développement d'un rapport interactif avec Streamlit :
    * Création du rapport : utiliser Streamlit pour développer un rapport interactif permettant de présenter les résultats de l'analyse et de la modélisation de manière dynamique et accessible. 
    * Présentation des résultats : permettre aux utilisateurs d'explorer les résultats et les insights via une interface web simple, où ils pourront interagir avec les données, filtrer les résultats et visualiser les performances des modèles en temps réel.

##### Dimension Économique

**Amélioration du processus de prise de décision stratégique :** l’utilisation d’un modèle de machine learning permet une analyse facilitée des données dans le but de formuler des recommandations stratégiques pour l’optimisation des résultats des campagnes futures. Cela soutient une prise de décision plus éclairée, permettant de saisir de nouvelles opportunités économiques et limiter les coût financiers d’une campagne marketing.

**Innovation pour la compétitivité :** en introduisant de nouvelles approches analytiques et en optimisant les processus, le projet contribue à l'innovation continue. Cela renforce la compétitivité de l'entreprise qui adopte des pratiques de gestion plus efficaces et améliore la réponse aux évolutions du marché.

#### Objectifs

Nous disposons de données personnelles sur les prospects de la banque ayant été contactés pour souscrire à un "dépôt à terme", ainsi que les résultats de ces opérations (souscription ou non du dépôt à terme suite au télémarketing).

Notre objectif est de prédire le succès d'une campagne marketing en déterminant à l'avance si un client est susceptible de souscrire au produit. Cette prédiction permettra d'optimiser le déploiement de la campagne, en maximisant le taux de souscriptions par rapport au nombre de prospections réalisées.

Pour atteindre cet objectif, nous devrons :
* Cibler les prospects les plus susceptibles de souscrire à un dépôt à terme en établissant un profil type afin d'éviter les prospections inefficaces.
* Déterminer l'approche la plus efficace en optimisant la stratégie de contact pour améliorer les taux de conversion


### COMPREHENSION ET MANIPULATION DES DONNEES

In [61]:
display(Markdown(txt_cadre))

#### Cadre

Le jeu de données dont nous disposons est accessible librement sur la plateforme Kaggle : https://www.kaggle.com/datasets/janiobachmann/bank-marketing-dataset/data

Ces données sont sous License CC0: Public Domain

La volumétrie du jeu de données est de 17 colonnes pour 11 162 lignes.

Les données sont de différentes natures (7 entiers, 6 chaînes de caractères, 4 binaires), et se décomposent en 6 variables quantitatives (age, balance, duration, campaign, pdays, previous) et 11 variables catégorielles (job, marital, education, default, housing, loan, contact, day, month, poutcome, deposit).


In [42]:
# Création d'un dataframe pour lire le data set
df_bank = pd.read_csv(data_path / 'bank.csv', sep = ",")

In [43]:
# Exploration macro du dataframe

print("Apercu du dataframe issu du data set bank :")
display(df_bank.head())

Apercu du dataframe issu du data set bank :


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


In [44]:
# Inspection des données du dataframe

print("Apercu de la structure du dataframe :")
print(df_bank.info())

Apercu de la structure du dataframe :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  object
 2   marital    11162 non-null  object
 3   education  11162 non-null  object
 4   default    11162 non-null  object
 5   balance    11162 non-null  int64 
 6   housing    11162 non-null  object
 7   loan       11162 non-null  object
 8   contact    11162 non-null  object
 9   day        11162 non-null  int64 
 10  month      11162 non-null  object
 11  duration   11162 non-null  int64 
 12  campaign   11162 non-null  int64 
 13  pdays      11162 non-null  int64 
 14  previous   11162 non-null  int64 
 15  poutcome   11162 non-null  object
 16  deposit    11162 non-null  object
dtypes: int64(7), object(10)
memory usage: 1.4+ MB
None


Les types de variables associés à chaque colonne sont en accord avec ce que représentent ces colonnes.

In [63]:
display(Markdown(txt_pertinence))

#### Pertinence

La variable la plus importante est la variable binaire « deposit » qui indique si le prospect a finalement souscrit ou non un dépôt à terme suite à la campagne marketing.

Nous devons donc mettre en place un modèle d’apprentissage supervisé suivant la technique de la classification (prédiction d’une variable cible de type qualitatif).

Il nous appartiendra de déterminer la corrélation entre cette variable « deposit » et les autres variables du dataset, pour pouvoir la prédire au final.


#### Pre-processing et feature engineering

In [45]:
print("Nombre de valeurs en doublon dans le dataframe :", df_bank.duplicated().sum())
print("\n")

print("Nombre de valeurs manquantes par colonne :")
print(df_bank.isna().sum())
print("\n")

print("Description des variables quantitatives :")
display(df_bank.describe())
print("\n")

print("Valeurs prises par les différentes variables catégorielles :")
print("\n")
for column in df_bank.select_dtypes(include='object').columns:
    print("- " + column + " :")
    print(df_bank[column].unique())
    print("\n")

Nombre de valeurs en doublon dans le dataframe : 0


Nombre de valeurs manquantes par colonne :
age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
deposit      0
dtype: int64


Description des variables quantitatives :


,age,balance,day,duration,campaign,pdays,previous
count,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000,11162.000000
mean,41.231948,1528.538524,15.658036,371.993818,2.508421,51.330407,0.832557
std,11.913369,3225.413326,8.420740,347.128386,2.722077,108.758282,2.292007
min,18.000000,-6847.000000,1.000000,2.000000,1.000000,-1.000000,0.000000
25%,32.000000,122.000000,8.000000,138.000000,1.000000,-1.000000,0.000000
50%,39.000000,550.000000,15.000000,255.000000,2.000000,-1.000000,0.000000
75%,49.000000,1708.000000,22.000000,496.000000,3.000000,20.750000,1.000000
max,95.000000,81204.000000,31.000000,3881.000000,63.000000,854.000000,58.000000




Valeurs prises par les différentes variables catégorielles :


- job :
['admin.' 'technician' 'services' 'management' 'retired' 'blue-collar'
 'unemployed' 'entrepreneur' 'housemaid' 'unknown' 'self-employed'
 'student']


- marital :
['married' 'single' 'divorced']


- education :
['secondary' 'tertiary' 'primary' 'unknown']


- default :
['no' 'yes']


- housing :
['yes' 'no']


- loan :
['no' 'yes']


- contact :
['unknown' 'cellular' 'telephone']


- month :
['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'jan' 'feb' 'mar' 'apr' 'sep']


- poutcome :
['unknown' 'other' 'failure' 'success']


- deposit :
['yes' 'no']




Le dataframe ne contient aucune valeur manquante à proprement parler mais comprends des champs renseignées unknown à retraiter.

In [46]:
# Insérer les calculs des taux de unknown






#########################################

#### Répartition des modalités pour chacune des colonnes :

In [47]:
var_num = ["age","balance","duration","campaign","pdays","previous"]
var_cat = ["job","marital","education","default","housing","loan","contact","day","month","poutcome","deposit"]

def creation_png_deposit(var, histo_var_deposit):
    full_path = build_path / f"graph_histo_deposit_{var}.png"
    histo_var_deposit.write_image(full_path)
    print("Graphique enregistré sous", full_path)

# Boxplot de distribution des variables quantitatives
for var in var_num:
    print("\n")
    print("Variable ", var)
    fig_num = go.Figure()
    fig_num.add_trace(go.Box(
        x=df_bank[var],
        name=var,
        marker_color="#222A2A",
        opacity=0.7
        ))
    fig_num.update_layout(
        title=("Distribution de la variable " + var),
        plot_bgcolor="rgba(0,0,0,0)",
        xaxis=dict(showgrid=False),
        yaxis=dict(gridcolor="rgba(210,210,210,0.5)")
        )
#    fig_num.show("notebook") 
    full_path = build_path / f"graph_distrib_var_num_{var}.png"
    fig_num.write_image(full_path)
    print("Graphique enregistré sous", full_path)
    if var == "age":
        print("Distribution cohérente, concentrée autour de la médiane, pas de valeurs aberrantes.")
#       Histogramme age / deposit
        fig_var_num_deposit = go.Figure()
        fig_var_num_deposit.add_trace(go.Histogram(
            x=df_bank[df_bank["deposit"] == "no"][var],
            name="No",
            marker_color="#222A2A",
            opacity=0.7,
            nbinsx=75
            ))
        fig_var_num_deposit.add_trace(go.Histogram(
            x=df_bank[df_bank["deposit"] == "yes"][var],
            name="Yes",
            marker_color="#19D3F3",
            opacity=0.7,
            nbinsx=75
            ))
        fig_var_num_deposit.update_layout(
            title="Souscription au dépôt selon la variable " + var,
            xaxis_title=var,
            yaxis_title="Nombre de clients",
            barmode="group",
            legend_title="Souscription",
            plot_bgcolor="rgba(0,0,0,0)",
            xaxis=dict(showgrid=False),
            yaxis=dict(gridcolor="rgba(210,210,210,0.5)")
            )
#        fig_var_num_deposit.show("notebook")
        creation_png_deposit(var, fig_var_num_deposit)
        print("Nous ne créerons pas de catégories d’âge pour le modèle de Machine Learning. Nous pourrons néanmoins les prévoir pour établir un profil type de clients susceptibles de souscrire un dépôt à terme.")
        print("Nous pouvons d’ores et déjà constater que le ratio souscriptions/non-souscriptions est en faveur des prospects âgés de moins de 29 ans ou plus de 60 ans.")
    if var == "balance":
        print("Distribution cohérente, concentrée autour de la médiane, pas de valeurs aberrantes.")
#       Histogramme balance / deposit
        fig_var_num_deposit = go.Figure()
        fig_var_num_deposit.add_trace(go.Histogram(
            x=df_bank[(df_bank["deposit"] == "no") & (df_bank["balance"] > -800) & (df_bank["balance"] < 4000)][var],
            name="No",
            marker_color="#222A2A",
            opacity=0.7,
            nbinsx=25
            ))
        fig_var_num_deposit.add_trace(go.Histogram(
            x=df_bank[(df_bank["deposit"] == "yes") & (df_bank["balance"] > -800) & (df_bank["balance"] < 4000)][var],
            name="Yes",
            marker_color="#19D3F3",
            opacity=0.7,
            nbinsx=25
            ))
        fig_var_num_deposit.update_layout(
            title="Souscription au dépôt selon la variable " + var,
            xaxis_title=var,
            yaxis_title="Nombre de clients",
            barmode="group",
            legend_title="Souscription",
            plot_bgcolor="rgba(0,0,0,0)",
            xaxis=dict(showgrid=False),
            yaxis=dict(gridcolor="rgba(210,210,210,0.5)")
            )
#        fig_var_num_deposit.show("notebook")
        creation_png_deposit(var, fig_var_num_deposit)
        print("Nous ne créerons pas de catégories pour le modèle de Machine Learning. Nous pourrons néanmoins les prévoir pour étudier un profil type de clients susceptibles de souscrire un dépôt à terme.")
        print("Le ratio souscriptions/non-souscriptions est négatif pour les clients dont le solde bancaire est négatif ou faible (inférieur à 800 euros), ce qui parait plutôt cohérent.")
    if var == "duration":
        print("En théorie, pour le Machine Learning, une variable doit être connue a priori, ce qui n’est pas le cas de la variable distribution. Nous verrons par la suite, lors de l’interprétation de nos modèles, si cette variable est importante ou non pour la prédiction.")
        print("- Si elle figure dans le top 5 des variables utilisées par le modèle, nous la conserverons.") 
        print("- Sinon, nous la supprimerons pour le Machine Learning, mais nous la conserverons pour émettre des recommandations métier. Cette information peut être utilisée pour valoriser l’intérêt suscité chez le client lors de la campagne passée et l’exploiter pour des prospections futures (plus l’appel était long, plus le client semblait intéressé et donc potentiel prospect pour la prochaine campagne)")
    if var == "campaign":
#       Histogramme campaign / deposit
        fig_var_num_deposit = go.Figure()
        fig_var_num_deposit.add_trace(go.Histogram(
            x=df_bank[df_bank["deposit"] == "no"][df_bank["campaign"] <= 8][var],
            name="No",
            marker_color="#222A2A",
            opacity=0.7,
            nbinsx=8
            ))
        fig_var_num_deposit.add_trace(go.Histogram(
            x=df_bank[df_bank["deposit"] == "yes"][df_bank["campaign"] <= 8][var],
            name="Yes",
            marker_color="#19D3F3",
            opacity=0.7,
            nbinsx=8
            ))
        fig_var_num_deposit.update_layout(
            title="Souscription au dépôt selon le nombre d'appels",
            xaxis_title="Nombre d'appels",
            yaxis_title="Nombre de clients",
            barmode="group",
            legend_title="Souscription",
            plot_bgcolor="rgba(0,0,0,0)",
            xaxis=dict(showgrid=False),
            yaxis=dict(gridcolor="rgba(210,210,210,0.5)")
            )
#        fig_var_num_deposit.show("notebook")
        creation_png_deposit(var, fig_var_num_deposit)
        print("Cette information peut être utilisée en complément d’un travail de profiling, pour adapter la méthode d’approche. En l’occurrence, le ratio gain (souscription) / perte (effort fourni pour le démarchage) semble ne plus être intéressant au-delà d’un appel.")
    if var == "pdays":
#       Histogramme pdays / deposit
        fig_var_num_deposit = go.Figure()
        fig_var_num_deposit.add_trace(go.Histogram(
            x=df_bank[df_bank["deposit"] == "no"][var],
            name="No",
            marker_color="#222A2A",
            opacity=0.7,
            nbinsx=10
            ))
        fig_var_num_deposit.add_trace(go.Histogram(
            x=df_bank[df_bank["deposit"] == "yes"][var],
            name="Yes",
            marker_color="#19D3F3",
            opacity=0.7,
            nbinsx=10
            ))
        fig_var_num_deposit.update_layout(
            title="Souscription au dépôt selon le nombre de jours depuis la dernière campagne",
            xaxis_title="Nombre de jours depuis la dernière campagne",
            yaxis_title="Nombre de clients",
            barmode="group",
            legend_title="Souscription",
            plot_bgcolor="rgba(0,0,0,0)",
            xaxis=dict(showgrid=False),
            yaxis=dict(gridcolor="rgba(210,210,210,0.5)")
            )
#        fig_var_num_deposit.show("notebook")
        creation_png_deposit(var, fig_var_num_deposit)
        print(separation)
        print("Valeurs prises par pdays quand poutcome est unknown :\n", (df_bank.loc[df_bank["poutcome"] == "unknown"]["pdays"].value_counts()))
        print(separation)
        print("Valeurs prises par poutcome quand pdays est -1 :\n", (df_bank.loc[df_bank["pdays"] == -1]["poutcome"].value_counts()))
        print(separation)
        print("Il y a beaucoup de lignes dont pdays est égal à -1. Pour tous les pdays à -1, poutcome est « unknown » (8 324 valeurs). Pour poutcome à « unknown » (8 863 valeurs au total), pdays varie entre -1 et 391.")
        print("Nous posons le postulat de départ suivant : pdays à -1 et donc par déduction poutcome à « unknown » indique que le client n’a jamais été contacté auparavant pour une campagne précédente. Il s’agit de nouveaux prospects.")
    if var == "previous":
#       Histogramme previous / deposit
        fig_var_num_deposit = go.Figure()
        fig_var_num_deposit.add_trace(go.Histogram(
            x=df_bank[df_bank["deposit"] == "no"][df_bank["previous"]<10][var],
            name="No",
            marker_color="#222A2A",
            opacity=0.7,
            nbinsx=10
            ))
        fig_var_num_deposit.add_trace(go.Histogram(
            x=df_bank[df_bank["deposit"] == "yes"][df_bank["previous"]<10][var],
            name="Yes",
            marker_color="#19D3F3",
            opacity=0.7,
            nbinsx=10
            ))
        fig_var_num_deposit.update_layout(
            title="Souscription au dépôt selon le nombre de contacts pré-campagne",
            xaxis_title="Nombre de contacts pré-campagne",
            yaxis_title="Nombre de clients",
            barmode="group",
            legend_title="Souscription",
            plot_bgcolor="rgba(0,0,0,0)",
            xaxis=dict(showgrid=False),
            yaxis=dict(gridcolor="rgba(210,210,210,0.5)")
            )
#        fig_var_num_deposit.show("notebook")
        creation_png_deposit(var, fig_var_num_deposit)
        print("Une grande partie des souscriptions lors de cette campagne ont été réalisées par des nouveaux prospects. Néanmoins, le ratio souscription/non-souscription est plus intéressant pour les clients ayant déjà été contactés lors de précédentes campagnes marketing.")



Variable  age
Graphique enregistré sous build/graph_distrib_var_num_age.png
Distribution cohérente, concentrée autour de la médiane, pas de valeurs aberrantes.
Graphique enregistré sous build/graph_histo_deposit_age.png
Nous ne créerons pas de catégories d’âge pour le modèle de Machine Learning. Nous pourrons néanmoins les prévoir pour établir un profil type de clients susceptibles de souscrire un dépôt à terme.
Nous pouvons d’ores et déjà constater que le ratio souscriptions/non-souscriptions est en faveur des prospects âgés de moins de 29 ans ou plus de 60 ans.


Variable  balance
Graphique enregistré sous build/graph_distrib_var_num_balance.png
Distribution cohérente, concentrée autour de la médiane, pas de valeurs aberrantes.
Graphique enregistré sous build/graph_histo_deposit_balance.png
Nous ne créerons pas de catégories pour le modèle de Machine Learning. Nous pourrons néanmoins les prévoir pour étudier un profil type de clients susceptibles de souscrire un dépôt à terme.
Le r

/tmp/ipykernel_13041/64204593.py:102: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/tmp/ipykernel_13041/64204593.py:109: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



Graphique enregistré sous build/graph_histo_deposit_campaign.png
Cette information peut être utilisée en complément d’un travail de profiling, pour adapter la méthode d’approche. En l’occurrence, le ratio gain (souscription) / perte (effort fourni pour le démarchage) semble ne plus être intéressant au-delà d’un appel.


Variable  pdays
Graphique enregistré sous build/graph_distrib_var_num_pdays.png
Graphique enregistré sous build/graph_histo_deposit_pdays.png
--------------------------------------------------------
Valeurs prises par pdays quand poutcome est unknown :
 pdays
-1      8324
 188       1
 98        1
Name: count, dtype: int64
--------------------------------------------------------
Valeurs prises par poutcome quand pdays est -1 :
 poutcome
unknown    8324
Name: count, dtype: int64
--------------------------------------------------------
Il y a beaucoup de lignes dont pdays est égal à -1. Pour tous les pdays à -1, poutcome est « unknown » (8 324 valeurs). Pour poutcome à « 

/tmp/ipykernel_13041/64204593.py:168: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/tmp/ipykernel_13041/64204593.py:175: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [48]:
jobs_education = {}
for element in df_bank["job"].unique():
    selection_job = df_bank.loc[df_bank["job"]== element]
    valeurs_job = selection_job["education"].value_counts()
    mode_job = selection_job["education"].mode()[0]
    jobs_education[element] = mode_job
    
# Histogramme de distribution des variables catégorielles
for var in var_cat:
    print("\n")
    print("Variable ", var)
#   Histogramme variable catégorielle / deposit
    if var not in ("day", "month"):
        fig_var_cat_deposit = go.Figure()
        fig_var_cat_deposit.add_trace(go.Histogram(
            x=df_bank[df_bank["deposit"] == "no"][var],
            name="No",
            marker_color="#222A2A",
            opacity=0.7,
            nbinsx=12
            ))
        fig_var_cat_deposit.add_trace(go.Histogram(
            x=df_bank[df_bank["deposit"] == "yes"][var],
            name="Yes",
            marker_color="#19D3F3",
            opacity=0.7,
            nbinsx=12
            ))
        fig_var_cat_deposit.update_layout(
            title="Souscription au dépôt selon la variable " + var,
            xaxis_title=var,
            yaxis_title="Nombre de clients",
            barmode="group",
            legend_title="Souscription",
            plot_bgcolor="rgba(0,0,0,0)",
            xaxis=dict(showgrid=False),
            yaxis=dict(gridcolor="rgba(210,210,210,0.5)")
            )
#        fig_var_cat_deposit.show("notebook")
        creation_png_deposit(var, fig_var_cat_deposit)
    if var == "job":
        print("La variable job comprend très peu de valeurs « unknown » (70 lignes pour une volumétrie totale de plus de 11 000 lignes). Etant donné le faible impact de ces lignes, nous pouvons simplement les supprimer.")
        print("Les retraités et étudiants semblent le plus sensibles à la question du dépôt à terme. Cela conforte notre analyse basée sur l’âge (moins de 29 ans et plus de 60 ans).")
    if var == "education":
        print("Les clients issus d’études tertiaires semblent plus intéressés par la souscription d’un dépôt à terme.")
#       Répartition job par niveau d'éducation
        colors = ["#19D3F3", "#4B4B4B", "#1E90FF", "#060808"]
        category_order = ["primary", "secondary", "tertiary", "unknown"]
        fig_educ_job = go.Figure()
        for i, education in enumerate(category_order):
            if education in df_bank["education"].unique():
                fig_educ_job.add_trace(go.Histogram(
                x=df_bank[df_bank["education"] == education]["job"],
                name=education,
                marker_color=colors[i],
                opacity=0.7
                ))
        fig_educ_job.update_layout(
            title="Distribution de la variable education par job",
            xaxis_title="Job",
            yaxis_title="Nombre de clients",
            barmode="group",
            plot_bgcolor="rgba(0,0,0,0)",
            xaxis=dict(showgrid=False),
            yaxis=dict(gridcolor="rgba(210,210,210,0.5)"),
            showlegend=True
            )
#        fig_educ_job.show("notebook")
        full_path = build_path / f"graph_histo_educ_par_job.png"
        fig_educ_job.write_image(full_path)
        print("Graphique enregistré sous", full_path)
        print("Les variables job et educations semblent corrélées entre elles. Pour chaque job, une donnée education semble ressortir clairement de manière générale.")
        print("Le résultat est plus mitigé pour les retraités et entrepreneurs, mais le nombre de clients issus de ces catégories semble suffisamment faible pour pouvoir en faire abstraction.")
        print("Nous pouvons donc en conclure que la variable education, lorsqu’elle est manquante dans le dataset, peut être déduite de la variable job.")
        print("Modalités les plus fréquentes de education par job :")
        print(jobs_education)
    if var == "marital":
        print("Les clients mariés semblent moins enclins à souscrire un dépôt à terme suite à une campagne marketing. Les célibataires seront de meilleures cibles.")
    if var == "default":
            print("Il y a un fort déséquilibre dans la répartition des données : très peu de clients ayant un crédit en défaut sont présents dans le dataset.")
    if var == "housing":
        print("Ne pas avoir de crédit immobilier en cours semble favoriser la souscription d’un dépôt à terme.")
    if var == "loan":
        print("Il a peu de clients ayant un crédit personnel en cours dans le dataset.")
        print("Néanmoins il semblerait qu’un client ayant un crédit personnel en cours soit moins enclin à souscrire un dépôt à terme.")
    if var == "contact":
        print("Proportion largement majoritaire de téléphones mobiles, la catégorie « unknown » emporte beaucoup de non-souscriptions. Dans une société où le téléphone portable prime largement sur la ligne fixe et où un certain nombre de ménages ne disposent même pas d’un téléphone fixe et fonctionnent uniquement par téléphone portable, on peut se poser la question de la pertinence de cette variable. Nous faisons le choix de la supprimer du dataset.")
    if var == "day":
#       Histogramme deposit / jour de l'appel
        fig_day_deposit = go.Figure()
        fig_day_deposit.add_trace(go.Histogram(
            x=df_bank[df_bank["deposit"] == "no"][var],
            name="No",
            marker_color="#222A2A",
            opacity=0.7,
            nbinsx=31
            ))
        fig_day_deposit.add_trace(go.Histogram(
            x=df_bank[df_bank["deposit"] == "yes"][var],
            name="Yes",
            marker_color="#19D3F3",
            opacity=0.7,
            nbinsx=31
            ))
        fig_day_deposit.update_layout(
            title="Souscription au dépôt selon le jour de l'appel",
            xaxis_title="Jour de l'appel dans le mois",
            yaxis_title="Nombre de clients",
            barmode="group",
            legend_title="Souscription",
            plot_bgcolor="rgba(0,0,0,0)",
            xaxis=dict(showgrid=False),
            yaxis=dict(gridcolor="rgba(210,210,210,0.5)")
            )
#        fig_day_deposit.show("notebook")
        creation_png_deposit(var, fig_var_cat_deposit)
        print("Certains jours du mois semblent plus favorables au démarchage : les 4 premiers jours du mois, le 10 et le 30 de chaque mois.")
    if var == "month":
#       Histogramme deposit / mois de l'appel
#       Création d'un ordre calendaire pour clarifier le graphique suivant
        month_order = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
        df_bank['month'] = pd.Categorical(df_bank['month'], categories=month_order, ordered=True)
        df_bank['month_numeric'] = df_bank['month'].cat.codes
        fig_month_deposit = go.Figure()
        fig_month_deposit.add_trace(go.Histogram(
            x=df_bank[df_bank["deposit"] == "no"]['month_numeric'],
            name="No",
            marker_color="#222A2A",
            opacity=0.7,
            nbinsx=len(month_order)
            ))
        fig_month_deposit.add_trace(go.Histogram(
            x=df_bank[df_bank["deposit"] == "yes"]['month_numeric'],
            name="Yes",
            marker_color="#19D3F3",
            opacity=0.7,
            nbinsx=len(month_order)
            ))
        fig_month_deposit.update_layout(
            title="Souscription au dépôt selon le mois de l'appel",
            xaxis_title="Mois de l'appel",
            yaxis_title="Nombre de clients",
            barmode="group",
            legend_title="Souscription",
            plot_bgcolor="rgba(0,0,0,0)",
            yaxis=dict(gridcolor="rgba(210,210,210,0.5)"),
            xaxis=dict(showgrid=False, 
                tickvals=list(range(len(month_order))),
                ticktext=month_order
                )
            )
#        fig_month_deposit.show("notebook")
        creation_png_deposit(var, fig_var_cat_deposit)
        print("On peut d’ores et déjà constater que les mois les plus propices au démarchage sont mars, avril, septembre, octobre, décembre. Au contraire, les démarchages ayant lieu en mai donnent de très mauvais retours. Ce mois est celui qui fait l'objet du plus grand nombre de prospections pendant cette campagne, tout en ayant le taux de conversion le plus faible de l'année.")
    if var == "poutcome":
        print("Il y a beaucoup de clients pour lesquels nous ne connaissons pas l’issue de la campagne précédente. D’après notre analyse de la variable pdays combinée à la variable poutcome, poutcome à « unknown » nous donne une information à part entière : il s’agit là de nouveaux prospects, jamais contactés auparavant. Nous décidons donc de conserver les 4 catégories de valeurs telles qu’elles sont dans la base de données initiale.")
    if var == "deposit":
        print("La variable deposit est notre variable cible. Sa distribution est très équilibrée, et elle ne comprend aucune valeur manquante. Elle ne nécessite donc aucun retraitement.")



Variable  job
Graphique enregistré sous build/graph_histo_deposit_job.png
La variable job comprend très peu de valeurs « unknown » (70 lignes pour une volumétrie totale de plus de 11 000 lignes). Etant donné le faible impact de ces lignes, nous pouvons simplement les supprimer.
Les retraités et étudiants semblent le plus sensibles à la question du dépôt à terme. Cela conforte notre analyse basée sur l’âge (moins de 29 ans et plus de 60 ans).


Variable  marital
Graphique enregistré sous build/graph_histo_deposit_marital.png
Les clients mariés semblent moins enclins à souscrire un dépôt à terme suite à une campagne marketing. Les célibataires seront de meilleures cibles.


Variable  education
Graphique enregistré sous build/graph_histo_deposit_education.png
Les clients issus d’études tertiaires semblent plus intéressés par la souscription d’un dépôt à terme.
Graphique enregistré sous build/graph_histo_educ_par_job.png
Les variables job et educations semblent corrélées entre elles. Pou

Récapitulatif des informations dont nous disposons sur les différentes variables du dataset :

In [49]:
donnees_path = pages_path / '2-donnees'

df_pertinence = pd.read_csv(donnees_path / 'tableau_variables.csv', sep = ";", index_col=0, lineterminator="\n")
display(df_pertinence)

,Description,Type info.,Taux de NaN,Distribution des valeurs,Type de variable,Commentaire / Retraitements à réaliser
Nom de la colonne,,,,,,
age,âge du client lors du dépôt,int64,0 %,de 18 à 95 ans avec une concentration entre 32...,quantitative,Distribution cohérente mais très concentrée au...
job,métier pratiqué,object,1 %,['admin.' 'technician' 'services' 'management'...,catégorielle,"70 lignes « unknown », impact dérisoire étant ..."
marital,statut marital,object,0 %,['married' 'single' 'divorced'],catégorielle,NaN
education,niveau d'études,object,4 %,['secondary' 'tertiary' 'primary' 'unknown'],catégorielle,497 lignes « unknown » ;\nContrôle des valeurs...
default,le crédit est-il en défaut ?,object,0 %,['yes' 'no'],catégorielle,Très peu de défauts de crédit dans le dataset
balance,solde bancaire annuel moyen,int64,0 %,de -6847 à 81 204 euros avec une concentration...,quantitative,"Distribution concentrée autour de la médiane, ..."
housing,le client bénéficie-t-il d’un crédit immobilier ?,object,0 %,['yes' 'no'],catégorielle,NaN
loan,le client bénéficie-t-il d’un prêt personnel ?,object,0 %,['yes' 'no'],catégorielle,Très peu de prêts personnels dans le dataset
contact,moyen de contacter le client,object,21 %,['unknown' 'cellular' 'telephone'],catégorielle,Donnée peu pertinente : proportion beaucoup pl...


In [64]:
display(Markdown(txt_conclusion_prepocess))

#### Conclusions sur les variables du dataset

Nous disposons d’ores et déjà de certaines informations nous permettant de cibler nos futurs prospects. Il semblerait que notre prospect idéal soit :
* âgé de moins de 29 ans ou plus de 60 ans,
* célibataire,
* étudiant ou retraité
* issu un cursus d’études du domaine tertiaire
* ayant quelques économies personnelles
* n’ayant aucun crédit en cours
* ayant déjà été contacté lors d’une campagne précédente mais ayant eu lieu il y a moins de 200 jours
* ayant souscrit un produit lors de la campagne précédente.

Nos analyses macro font également ressortir quelques recommandations métier. Les méthodes d’approche à privilégier sont les suivantes :
* contact par appel téléphonique sur une ligne mobile
* avant le 5 du mois ou alors le 10 ou le 30 du mois
* en mars, avril, septembre, octobre ou décembre
* un seul appel par prospect

Pour affiner nos analyses et réussir à prédire si un prospect serait susceptible de souscrire ou non à un dépôt à terme, nous devons passer à l’étape de la modélisation. Néanmoins, avant cela, nous devons procéder aux retraitements suivants :
* age : créer des catégories pour faciliter le traitement
* job : 1% de valeurs "unknown"(soit 70 lignes), nous avons pris la décision de supprimer ces lignes qui sont peu nombreuses
* education : 4% de valeurs "unknown" (soit 497 lignes), cette variable est importante et ces lignes sont trop nombreuses pour être supprimées. Nous avons donc décidé de compléter les valeurs manquantes. La variable education étant fortement corrélée avec la variable job, nous déduirons les données manquantes du job associé (modalité education la plus fréquente pour le job en question)
* contact : 21% de valeurs "unknown". Après analyse nous avons décidé de supprimer cette colonne qui ne représente pas d'intérêt majeur pour la modélisation.

Pour rappel : la variable poutcome restera inchangée.
En effet, cette variable comporte trop de valeurs manquantes pour pouvoir supprimer des lignes (75% de valeurs "unknown").
De plus, nous avons acté qu'elle était importante pour notre modélisation.
La valeur "unknown" de cette variable constituera donc une valeur à part entière dans notre projet.


### MODELISATION DU PROJET

In [68]:
display(Markdown(txt_classif_choix))

#### Classification du problème

Comme nous l’avons vu précédemment, la variable la plus importante de notre jeu de données est la variable binaire « deposit » qui indique si le prospect a finalement souscrit ou non un dépôt à terme suite à la campagne marketing. Après les différents retraitements opérés lors de la phase de pré-processing, nous disposons de 15 variables explicatives et d’1 variable cible de nature catégorielle.

Nous devons donc mettre en place un modèle d’apprentissage supervisé suivant la technique de la classification (prédiction d’une variable cible de type qualitatif).

Nous avons choisi de nettoyer le dataset avant la modélisation car il y avait peu de données à retraiter et que ces retraitement auront peu d'impact sur la suite de notre projet :
* suppression d'une colonne (contact),
* suppression de seulement 70 lignes pour valeur job manquante,
* et remplacement des 427 valeurs manquantes restantes d'education (70 des valeurs manquantes ayant été supprimées avec la suppression des lignes dont job était manquant).

Le dataset ne présentant pas un grand volume, nous ne réduirons pas sa dimension lors de la modélisation.

Nous diviserons les données en deux parties : étant donné que nous disposons d'un dataset d'un volume correct (plus de 10 000 entrées), 70% du dataset seront dédiés à l’entraînement et 30% du dataset seront dédiés à l’évaluation de notre modèle.

**Choix de la métrique de performance**

Selon la classification de notre projet (modèle supervisé de classification), nous pouvons retenir 4 principales métriques de performance : l’exactitude, la précision, le rappel et le score F1.

* **L'exactitude (accuracy en anglais)** évalue le taux de bonnes prédictions par rapport au nombre total de prédictions. Elle est facile à calculer, facile à interpréter, et résume la performance du modèle avec une valeur unique. Elle évalue mal les performances d’un modèle basé sur des données déséquilibrées, mais ce n'est pas le cas dans notre projet. Cependant, l’exactitude ne permet pas de faire des nuances entre les différentes prédictions et néglige de ce fait le coût des faux négatifs. Notre choix se portera donc sur une métrique plus précise.
* **La précision** est une métrique qui répond à la question : Parmi toutes les prédictions positives du modèle, combien sont de vrais positifs ? Un score de précision élevé nous informe que le modèle ne classe pas aveuglément toutes les observations comme positives. Cette métrique est utile lorsque le coût des faux positifs est élevé.
* **Le rappel (recall en anglais)** est une métrique qui quantifie la proportion d'observations réellement positives qui ont été correctement classifiées positives par le modèle (vrai positifs par rapport à la somme des vrais positifs et faux négatifs). Un score de rappel élevé nous informe que le modèle est capable de bien détecter les observations réellement positives. Cette métrique est utile lorsque le coût des faux négatifs est élevé.
* **Le score F1 (F1-score en anglais)** est une métrique qui permet de combiner la précision et le rappel, puisqu'elle correspond à leur moyenne harmonique.

C'est dernière métrique, **le score F1**, qui nous parait la plus adaptée pour évaluer nos différents modèles de Machine Learning.


#### Choix du modèle et optimisation

**Choix de l'encodage**

Notre jeu de données ne comporte plus de valeurs manquantes mais contient encore des données quantitatives extrêmes. Nous avons décidé de les conserver car ces informations restent intéressantes pour notre modèle. Nous ne pouvons pas normaliser ou standardiser les variables concernées car ces techniques sont sensibles aux valeurs extrêmes. Nous devrons en revanche tester s’il est pertinent de les mettre à l’échelle par la technique de Robust Scaling. Les variables concernées sont : age, balance, duration, campaign, pdays, previous.

Les variables catégorielles, quant à elles, devront être encodées de la manière suivante :
* job : OneHotEncoding
* marital : OneHotEncoding
* education : Ordinal Encoding ou OneHotEncoding
* default : OneHotEncoding
* housing : OneHotEncoding
* loan : OneHotEncoding
* day : OneHotEncoding
* month : OneHotEncoding
* poutcome : OneHotEncoding
* deposit : LabelEncoding

**Choix des modèles**

Les modèles que nous avons testé sont les suivants :
* Régression logistique (LogiticRegression)
* Machines à Vecteurs de Support (SVC)
* Méthode des K plus proches voisins (KNN - KNeighborsClassifier)
* Arbres de décision (DecisionTreeClassifier et DecisionTreeRegressor)
* Forêts aléatoires (RandomForestClassifier)
* Gradient Boosting (GradientBoostingClassifier)
* Extreme Gradient Boosting (XGBClassifier)
* CatBoosting (CatBoostClassifier)

**Tests réalisés**

Nous avons testé nos modèles avec différents paramètres :
* avec / sans la variable duration
* avec / sans mise à l'échelle des variables numériques (Robust Scaling)
* avec Ordinal Encoding / OneHotEncoding pour la variable education
* avec / sans optimisation des hyperparamètres

**Interprétabilité des modèles**

Nous avons tenté de faire ressortir les variables les plus importantes utilisées par nos différents modèles pour décider du sort de la variable duration. Cependant, nous avons été confrontés à une limite : seuls quelques uns de nos modèles disposent de l'attribut feature_importances_ ou coef_.

De ce fait, nous avons finalement décidé d'étudier leur interprétabilité avec SHAP (SHapley Additive exPlanations).

**Optimisation des hyperparamètres**

Pour optimiser les hyperparamètres, nous avons fait appel à la méthode GridSearch : celle-ci explore toutes les combinaisons possibles d'hyperparamètres spécifiés pour trouver les meilleurs réglages du modèle.

Nous avons testé l'optimisation des hyperparamètres sur la quasi-totalité des modèles sélectionnés. Ces tests ont été réalisés en local car ils sont relativement lourds selon les modèles. Pour éviter de surcharger la plateforme Streamlit et ainsi perdre en efficacité, nous ne présenterons l'optimisation des paramètres que pour les 3 modèles qui nous semblent être les plus performants, à savoir : CatBoosting, Extreme Gradient Boosting et Forêts aléatoires.


In [50]:
# RETRAITEMENTS POUR GENERER df_bank_0 (Base utilisée uniquement pour le profiling)

# Création d'un dataframe df_bank_0 copie de df_bank :
df_bank_0 = df_bank.copy()

# Création d'une fonction qui définit la catégorie d'âge sur la base de "age":
def get_categ(age):
	if age <= 31:
		categ = "extreme_bas"
	elif 31 < age <= 40:
		categ = "jeune"
	elif 30 < age <= 49:
		categ = "moins_jeune"
	elif 49 < age:
		categ = "extreme_haut"
	return categ
    
df_bank_0["age_categ"] = df_bank_0["age"].apply(get_categ)

# Export du dataframe df_bank_1 en fichier .csv
df_bank_0.to_csv(build_path / 'bank_0_profiling.csv', index=False, sep=',')

In [51]:
# RETRAITEMENTS POUR GENERER df_bank_1 (retraitements minimalistes)

# Suppression des lignes dont la valeur "job" est manquante :
df_bank_1 = df_bank.loc[df_bank["job"] != "unknown"]

# Remplacement des unknown de "education" par la modalité la plus fréquente rencontrée pour un "job" identique :
df_bank_1.loc[df_bank_1["education"] == "unknown", "education"] = df_bank_1.loc[df_bank_1["education"] == "unknown", "job"].map(jobs_education)		

print("Valeurs prises par la variable job après retraitement :\n", df_bank_1["job"].value_counts())
print(separation)

print("Valeurs prises par la variable education après retraitement :\n", df_bank_1["education"].value_counts())
print(separation)

# Suppression de la colonne "contact" :
df_bank_1 = df_bank_1.drop("contact", axis=1)

print("Apercu de la structure du dataframe df_bank_1 après retraitements :")
print(df_bank_1.info())

# Export du dataframe df_bank_1 en fichier .csv
df_bank_1.to_csv(build_path / 'bank_1.csv', index=False, sep=',')

Valeurs prises par la variable job après retraitement :
 job
management       2566
blue-collar      1944
technician       1823
admin.           1334
services          923
retired           778
self-employed     405
student           360
unemployed        357
entrepreneur      328
housemaid         274
Name: count, dtype: int64
--------------------------------------------------------
Valeurs prises par la variable education après retraitement :
 education
secondary    5815
tertiary     3775
primary      1502
Name: count, dtype: int64
--------------------------------------------------------
Apercu de la structure du dataframe df_bank_1 après retraitements :
<class 'pandas.core.frame.DataFrame'>
Index: 11092 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   age            11092 non-null  int64   
 1   job            11092 non-null  object  
 2   marital        11092 non-null  object  
 3   ed

In [52]:
# RETRAITEMENTS SUPPLEMENTAIRES POUR GENERER df_bank_2 (sans duration)

# Suppression de la colonne "duration" :
df_bank_2 = df_bank_1.drop("duration", axis=1)
# Export du dataframe df_bank_2 en fichier .csv
df_bank_2.to_csv(build_path / 'bank_2.csv', index=False, sep=',')

print("Apercu de la structure du dataframe df_bank_2 après retraitements :")
print(df_bank_2.info())

Apercu de la structure du dataframe df_bank_2 après retraitements :
<class 'pandas.core.frame.DataFrame'>
Index: 11092 entries, 0 to 11161
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   age            11092 non-null  int64   
 1   job            11092 non-null  object  
 2   marital        11092 non-null  object  
 3   education      11092 non-null  object  
 4   default        11092 non-null  object  
 5   balance        11092 non-null  int64   
 6   housing        11092 non-null  object  
 7   loan           11092 non-null  object  
 8   day            11092 non-null  int64   
 9   month          11092 non-null  category
 10  campaign       11092 non-null  int64   
 11  pdays          11092 non-null  int64   
 12  previous       11092 non-null  int64   
 13  poutcome       11092 non-null  object  
 14  deposit        11092 non-null  object  
 15  month_numeric  11092 non-null  int8    
dtypes: category(1

#### Test 1 : avec / sans la variable duration

##### Test 1-1 : avec la variable duration / sans Robust Scaling / Ordinal Encoding pour education / sans optimisation des hyperparamètres 

##### Test 1-2 : avec la variable duration / avec Robust Scaling / Ordinal Encoding pour education / sans optimisation des hyperparamètres 

Lors de l’exploration des données, nous nous posions la question de la conservation ou non de la variable « duration ».
Nous n'avons pas pu faire resortir les variables les plus importantes pour tous nos modèles car certains ne possèdent pas d'attribut feature_importances_ ou coef_. Néanmoins, la variable duration semble être dans le top 5 des variables les plus importantes pour au moins 7 de nos modèles. Nous pourrions donc en déduire qu'elle joue un rôle prépondérant dans notre projet de prédiction et que nous ne pouvons donc pas la supprimer de notre base. Néanmoins, nous avons décidé d'interpréter ce résultat différemment : cette variable n'étant pas connue a priori, la place qu'elle semble prendre dans la prédiction est trop importante. Nous prenons donc le parti de la supprimer pour notre modèle de prédiction.

##### Test 1-3 : sans la variable duration / sans Robust Scaling / Ordinal Encoding pour education / sans optimisation des hyperparamètres 

#### Test 2 : avec / sans mise à l'échelle des variables numériques (Robust Scaling)

##### Test 2-1 : sans la variable duration / sans Robust Scaling / Ordinal Encoding pour education / sans optimisation des hyperparamètres
cf. Test 1-3


##### Test 2-2 : sans la variable duration / avec Robust Scaling / Ordinal Encoding pour education / sans optimisation des hyperparamètres 

La mise à l'échelle par RobustScaling agit positivement sur les modèles suivants : Logistic Regression, SVM, KNN, Decision Tree Classifier, Decision Tree Regressor et Random Forest.
Elle est cependant sans incidence pour les modèles Gradient Boost, Extreme Gradient Boost et CatBoost.

#### Test 3 : avec Ordinal Encoding / OneHotEncoding pour la variable education

##### Test 3-1 : sans la variable duration / avec Robust Scaling / Ordinal Encoding pour education / sans optimisation des hyperparamètres
cf. Test 2-2 

##### Test 3-2 : sans la variable duration / avec Robust Scaling / OneHotEncoding pour education / sans optimisation des hyperparamètres 

Le fait d’encoder la variable education en Ordinal Encoding ou OneHotEncoding semble peu importer. Les résultats sont équivalents avec les 2 méthodes.

#### Test 4 : avec / sans optimisation des hyperparamètres

##### Test 4-1 : sans la variable duration / avec Robust Scaling / Ordinal Encoding pour education / sans optimisation des hyperparamètres
cf. Test 3-1 


##### Test 4-2 : sans la variable duration / avec Robust Scaling / Ordinal Encoding pour education / avec optimisation des hyperparamètres 

### CONCLUSION DU PROJET